In [ ]:
!pip install pydantic[email]
!pip install fastapi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.8 MB/s eta 0:00:00


### Validação básica usando Pydantic

In [ ]:
from enum import auto, IntFlag
from typing import Any

from pydantic import (
    BaseModel,
    EmailStr,
    Field,
    SecretStr,
    ValidationError,
)

# Define um conjunto de papéis (roles) utilizando IntFlag,
# permitindo combinações lógicas entre permissões (ex: Admin possui todas as permissões).
class Role(IntFlag):
    Author = auto()
    Editor = auto()
    Developer = auto()
    Admin = Author | Editor | Developer

# Define um modelo de dados do usuário usando o Pydantic.
# O modelo aplica validações automáticas ao receber dados de entrada.
class User(BaseModel):
    name: str = Field(examples=["Arjan"])
    email: EmailStr = Field(
        examples=["example@arjancodes.com"],
        description="The email address of the user",
        frozen=True,
    )
    # Passwod é secret, logo, quando printado, será mascarado automaticamente para proteger informações sensíveis.
    password: SecretStr = Field(
        examples=["Password123"], description="The password of the user"
    )
    role: Role = Field(default=None, description="The role of the user")

# Função responsável por validar um dicionário de dados e instanciar o modelo User.
# Em caso de erro de validação, printa as mensagens de erro.
def validate(data: dict[str, Any]) -> None:
    try:
        # Tenta validar e criar um objeto User a partir dos dados fornecidos.
        user = User.model_validate(data)
        print(user)
    except ValidationError as e:
        # Caso os dados não passem nas validações do modelo, exibe os erros.
        print("User is invalid")
        for error in e.errors():
            print(error)


# Função principal que simula o uso do modelo com dados "bons" e "ruins".
def main() -> None:
    good_data = {
        "name": "Arjan",
        "email": "example@arjancodes.com",
        "password": "Password123",
    }
    bad_data = {"email": "<bad data>", "password": "<bad data>"}

    validate(good_data)
    validate(bad_data)


if __name__ == "__main__":
    main()

name='Arjan' email='example@arjancodes.com' password=SecretStr('**********') role=None
User is invalid
{'type': 'missing', 'loc': ('name',), 'msg': 'Field required', 'input': {'email': '<bad data>', 'password': '<bad data>'}, 'url': 'https://errors.pydantic.dev/2.11/v/missing'}
{'type': 'value_error', 'loc': ('email',), 'msg': 'value is not a valid email address: An email address must have an @-sign.', 'input': '<bad data>', 'ctx': {'reason': 'An email address must have an @-sign.'}}


### Usando Regex para validar dados com o Pydantic

In [ ]:
import enum
import hashlib
import re
from typing import Any

from pydantic import (
    BaseModel,
    EmailStr,
    Field,
    field_validator,
    model_validator,
    SecretStr,
    ValidationError,
)

# Regex para validar senhas fortes: pelo menos 8 caracteres, 1 minúscula, 1 maiúscula e 1 número
VALID_PASSWORD_REGEX = re.compile(r"^(?=.*[a-z])(?=.*[A-Z])(?=.*\d).{8,}$")

# Regex para nomes com apenas letras e no mínimo 2 caracteres
VALID_NAME_REGEX = re.compile(r"^[a-zA-Z]{2,}$")


class Role(enum.IntFlag):
    Author = 1
    Editor = 2
    Admin = 4
    SuperAdmin = 8

# Modelo principal representando um usuário
class User(BaseModel):
    name: str = Field(examples=["Arjan"])
    email: EmailStr = Field(
        examples=["user@arjancodes.com"],
        description="The email address of the user",
        frozen=True, # Protege contra mutabilidade após instanciado
    )
    password: SecretStr = Field(
        examples=["Password123"], description="The password of the user"
    )
    role: Role = Field(
        default=None, description="The role of the user", examples=[1, 2, 4, 8]
    )

    # Validação de campo: nome deve conter apenas letras e pelo menos 2 caracteres
    @field_validator("name")
    @classmethod
    def validate_name(cls, v: str) -> str:
        if not VALID_NAME_REGEX.match(v):
            raise ValueError(
                "Name is invalid, must contain only letters and be at least 2 characters long"
            )
        return v


    # Validação de campo (antes da conversão): aceita role como int, str ou Role
    @field_validator("role", mode="before")
    @classmethod
    def validate_role(cls, v: int | str | Role) -> Role:
        op = {int: lambda x: Role(x), str: lambda x: Role[x], Role: lambda x: x}
        try:
            return op[type(v)](v)
        except (KeyError, ValueError):
            raise ValueError(
                f"Role is invalid, please use one of the following: {', '.join([x.name for x in Role])}"
            )

    # Validação de modelo (antes da criação): lógica que depende de múltiplos campos
    @model_validator(mode="before")
    @classmethod
    def validate_user(cls, v: dict[str, Any]) -> dict[str, Any]:
        # Checa se campos obrigatórios estão presentes
        if "name" not in v or "password" not in v:
            raise ValueError("Name and password are required")
        # Impede que a senha contenha o nome do usuário
        if v["name"].casefold() in v["password"].casefold():
            raise ValueError("Password cannot contain name")
        # Valida se a senha segue os critérios definidos no regex
        if not VALID_PASSWORD_REGEX.match(v["password"]):
            raise ValueError(
                "Password is invalid, must contain 8 characters, 1 uppercase, 1 lowercase, 1 number"
            )
        # Aplica hash SHA-256 à senha
        v["password"] = hashlib.sha256(v["password"].encode()).hexdigest()
        return v


# Função que valida um dicionário com dados do usuário, instanciando o modelo User
def validate(data: dict[str, Any]) -> None:
    try:
        user = User.model_validate(data)
        print(user)
    # Exibe todos os erros com contexto (campo, tipo, msg)
    except ValidationError as e:
        print("User is invalid:")
        print(e)


# Função principal que executa diferentes cenários de teste
def main() -> None:
    test_data = dict(
        good_data={
            "name": "Arjan",
            "email": "example@arjancodes.com",
            "password": "Password123",
            "role": "Admin",
        },
        bad_role={
            "name": "Arjan",
            "email": "example@arjancodes.com",
            "password": "Password123",
            "role": "Programmer",
        },
        bad_data={
            "name": "Arjan",
            "email": "bad email",
            "password": "bad password",
        },
        bad_name={
            "name": "Arjan<-_->",
            "email": "example@arjancodes.com",
            "password": "Password123",
        },
        duplicate={
            "name": "Arjan",
            "email": "example@arjancodes.com",
            "password": "Arjan123",
        },
        missing_data={
            "email": "<bad data>",
            "password": "<bad data>",
        },
    )

    for example_name, data in test_data.items():
        print(example_name)
        validate(data)
        print()


if __name__ == "__main__":
    main()

good_data
name='Arjan' email='example@arjancodes.com' password=SecretStr('**********') role=<Role.Admin: 4>

bad_role
User is invalid:
1 validation error for User
role
  Value error, Role is invalid, please use one of the following: Author, Editor, Admin, SuperAdmin [type=value_error, input_value='Programmer', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

bad_data
User is invalid:
1 validation error for User
  Value error, Password is invalid, must contain 8 characters, 1 uppercase, 1 lowercase, 1 number [type=value_error, input_value={'name': 'Arjan', 'email'...ssword': 'bad password'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

bad_name
User is invalid:
1 validation error for User
name
  Value error, Name is invalid, must contain only letters and be at least 2 characters long [type=value_error, input_value='Arjan<-_->', input_type=str]
    For further information visit https://

### Serialização de dados com o Pydantic

In [ ]:
import enum
import hashlib
import re
from typing import Any, Self
from pydantic import (
    BaseModel,
    EmailStr,
    Field,
    field_serializer,
    field_validator,
    model_serializer,
    model_validator,
    SecretStr,
)

VALID_PASSWORD_REGEX = re.compile(r"^(?=.*[a-z])(?=.*[A-Z])(?=.*\d).{8,}$")
VALID_NAME_REGEX = re.compile(r"^[a-zA-Z]{2,}$")


class Role(enum.IntFlag):
    User = 0
    Author = 1
    Editor = 2
    Admin = 4
    SuperAdmin = 8


class User(BaseModel):
    name: str = Field(examples=["Example"])
    email: EmailStr = Field(
        examples=["user@arjancodes.com"],
        description="The email address of the user",
        frozen=True,
    )
    password: SecretStr = Field(
        examples=["Password123"], description="The password of the user", exclude=True
    )
    role: Role = Field(
        description="The role of the user",
        examples=[1, 2, 4, 8],
        default=0,
        validate_default=True,
    )

    @field_validator("name")
    def validate_name(cls, v: str) -> str:
        if not VALID_NAME_REGEX.match(v):
            raise ValueError(
                "Name is invalid, must contain only letters and be at least 2 characters long"
            )
        return v

    @field_validator("role", mode="before")
    @classmethod
    def validate_role(cls, v: int | str | Role) -> Role:
        op = {int: lambda x: Role(x), str: lambda x: Role[x], Role: lambda x: x}
        try:
            return op[type(v)](v)
        except (KeyError, ValueError):
            raise ValueError(
                f"Role is invalid, please use one of the following: {', '.join([x.name for x in Role])}"
            )

    @model_validator(mode="before")
    @classmethod
    def validate_user_pre(cls, v: dict[str, Any]) -> dict[str, Any]:
        if "name" not in v or "password" not in v:
            raise ValueError("Name and password are required")
        if v["name"].casefold() in v["password"].casefold():
            raise ValueError("Password cannot contain name")
        if not VALID_PASSWORD_REGEX.match(v["password"]):
            raise ValueError(
                "Password is invalid, must contain 8 characters, 1 uppercase, 1 lowercase, 1 number"
            )
        v["password"] = hashlib.sha256(v["password"].encode()).hexdigest()
        return v

    @model_validator(mode="after")
    def validate_user_post(self, v: Any) -> Self:
        if self.role == Role.Admin and self.name != "Arjan":
            raise ValueError("Only Arjan can be an admin")
        return self

    # Serializador de campo: ao serializar como JSON, transforma o enum em string (ex: "Admin")
    @field_serializer("role", when_used="json")
    @classmethod
    def serialize_role(cls, v) -> str:
        return v.name

    # Serializador de modelo: personalizado para controlar saída padrão em JSON
    @model_serializer(mode="wrap", when_used="json")
    def serialize_user(self, serializer, info) -> dict[str, Any]:
        # Caso nenhum include/exclude seja definido, retorna apenas name e role (como string)
        if not info.include and not info.exclude:
            return {"name": self.name, "role": self.role.name}
        return serializer(self)


# Função principal para testar o modelo e diferentes formas de serialização
def main() -> None:
    data = {
        "name": "Arjan",
        "email": "example@arjancodes.com",
        "password": "Password123",
        "role": "Admin",
    }
    user = User.model_validate(data)
    if user:
        print(
            "The serializer that returns a dict:",
            user.model_dump(),
            sep="\n",
            end="\n\n",
        )
        print(
            "The serializer that returns a JSON string:",
            user.model_dump(mode="json"),
            sep="\n",
            end="\n\n",
        )
        print(
            "The serializer that returns a json string, excluding the role:",
            user.model_dump(exclude=["role"], mode="json"),
            sep="\n",
            end="\n\n",
        )
        print("The serializer that encodes all values to a dict:", dict(user), sep="\n")


if __name__ == "__main__":
    main()

The serializer that returns a dict:
{'name': 'Arjan', 'email': 'example@arjancodes.com', 'role': <Role.Admin: 4>}

The serializer that returns a JSON string:
{'name': 'Arjan', 'role': 'Admin'}

The serializer that returns a json string, excluding the role:
{'name': 'Arjan', 'email': 'example@arjancodes.com'}

The serializer that encodes all values to a dict:
{'name': 'Arjan', 'email': 'example@arjancodes.com', 'password': SecretStr('**********'), 'role': <Role.Admin: 4>}


### Utilidade mais prática - Pydantic & FastAPI

In [ ]:
from datetime import datetime
from typing import Optional
from uuid import uuid4

from fastapi import FastAPI
from fastapi.responses import JSONResponse
from fastapi.testclient import TestClient
from pydantic import BaseModel, EmailStr, Field, field_serializer, UUID4

app = FastAPI()


class User(BaseModel):
    model_config = {
        "extra": "forbid", # Proíbe campos desconhecidos na criação do modelo
    }
    __users__ = [] # Armazena todos os usuários em memória
    name: str = Field(..., description="Name of the user")
    email: EmailStr = Field(..., description="Email address of the user")
    friends: list[UUID4] = Field(
        default_factory=list, max_items=500, description="List of friends"
    ) # Lista limitada de até 500 amigos por usuário
    blocked: list[UUID4] = Field(
        default_factory=list, max_items=500, description="List of blocked users"
    ) # Lista de bloqueados, seguindo mesmo limite que friends
    signup_ts: Optional[datetime] = Field(
        default_factory=datetime.now, description="Signup timestamp", kw_only=True
    ) # Requer uso como keyword argument na construção do modelo
    id: UUID4 = Field(
        default_factory=uuid4, description="Unique identifier", kw_only=True
    )# ID único gerado automaticamente

    @field_serializer("id", when_used="json")
    def serialize_id(self, id: UUID4) -> str:
        return str(id) # Garante que o UUID seja serializado como string


# Rota para retornar todos os usuários cadastrados
@app.get("/users", response_model=list[User])
async def get_users() -> list[User]:
    return list(User.__users__)

# Rota para criar um novo usuário
@app.post("/users", response_model=User)
async def create_user(user: User):
    User.__users__.append(user)
    return user

# Rota para buscar um usuário pelo ID
@app.get("/users/{user_id}", response_model=User)
async def get_user(user_id: UUID4) -> User | JSONResponse:
    try:
        return next((user for user in User.__users__ if user.id == user_id))
    except StopIteration:
        return JSONResponse(status_code=404, content={"message": "User not found"})

# Teste simplificado para rodar no colab
def main():
    with TestClient(app) as client:
        for i in range(5):
            response = client.post(
                "/users",
                json={"name": f"User {i}", "email": f"example{i}@arjancodes.com"},
            )
            print(response.json())

        # Testa listar todos os usuários
        response = client.get("/users")
        print("\nGET /users result:")
        print(response.json())

        # Testa busca individual
        user_id = response.json()[0]["id"]
        response = client.get(f"/users/{user_id}")
        print(f"\nGET /users/{user_id} result:")
        print(response.json())

        # Testa erro de e-mail inválido
        response = client.post("/users", json={"name": "User X", "email": "not-an-email"})
        print("\nPOST /users com email inválido:")
        print(response.status_code, response.json())


if __name__ == "__main__":
    main()

{'name': 'User 0', 'email': 'example0@arjancodes.com', 'friends': [], 'blocked': [], 'signup_ts': '2025-04-03T15:42:59.800097', 'id': '544fdf26-e42b-4f42-9f54-76a56fa63522'}
{'name': 'User 1', 'email': 'example1@arjancodes.com', 'friends': [], 'blocked': [], 'signup_ts': '2025-04-03T15:42:59.802040', 'id': 'd514d0b9-6dcd-419d-bf75-712e0afd6861'}
{'name': 'User 2', 'email': 'example2@arjancodes.com', 'friends': [], 'blocked': [], 'signup_ts': '2025-04-03T15:42:59.803551', 'id': '7db6cb3c-b142-4800-8153-f1d0233cbb97'}
{'name': 'User 3', 'email': 'example3@arjancodes.com', 'friends': [], 'blocked': [], 'signup_ts': '2025-04-03T15:42:59.804893', 'id': 'f9ee8fa6-51c0-41bd-8521-5910b0f3cf18'}
{'name': 'User 4', 'email': 'example4@arjancodes.com', 'friends': [], 'blocked': [], 'signup_ts': '2025-04-03T15:42:59.806217', 'id': 'e0c31a70-78b7-4285-a2e7-2db31aba29bb'}

GET /users result:
[{'name': 'User 0', 'email': 'example0@arjancodes.com', 'friends': [], 'blocked': [], 'signup_ts': '2025-04-03